In [14]:
import psycopg2
import pandas as pd

# Conectar
conn = psycopg2.connect(
    host="rawg-db.c5gics8qchki.eu-north-1.rds.amazonaws.com",
    user="soraya",
    password="Soraya123!",
    database="rawg_db"
)

query = """
SELECT
    g.game_id::int,
    g.slug::text,
    g.name::text,
    g.released::date,
    g.tba::boolean,
    g.rating::float,
    g.rating_top::float,
    g.ratings_count::int,
    g.reviews_text_count::int,
    g.metacritic::float,
    g.playtime::int,
    g.suggestions_count::int,
    g.added::int,
    g.reviews_count::int,
    g.background_image::text,
    g.dominant_color::text,
    g.saturated_color::text,
    g.esrb_rating_id::int,
    g.website::text,
    -- Nuevas columnas con IDs
    ARRAY_AGG(DISTINCT gg.genre_id) AS genre_ids,
    ARRAY_AGG(DISTINCT gp.platform_id) AS platform_ids,
    ARRAY_AGG(DISTINCT gs.store_id) AS store_ids
FROM public.games g
LEFT JOIN public.game_genres gg ON g.game_id = gg.game_id
LEFT JOIN public.game_platforms gp ON g.game_id = gp.game_id
LEFT JOIN public.game_stores gs ON g.game_id = gs.game_id
GROUP BY g.game_id, g.slug, g.name, g.released, g.tba, g.rating, g.rating_top,
         g.ratings_count, g.reviews_text_count, g.metacritic, g.playtime,
         g.suggestions_count, g.added, g.reviews_count, g.background_image,
         g.dominant_color, g.saturated_color, g.esrb_rating_id, g.website
ORDER BY g.game_id ASC;
"""

df = pd.read_sql(query, conn)
conn.close()

df= df[(df['metacritic'].fillna(0) > 0) | (df['ratings_count'].fillna(0) > 0)]

df


C:\Users\csd\AppData\Local\Temp\ipykernel_3684\3111040968.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,game_id,slug,name,released,tba,rating,rating_top,ratings_count,reviews_text_count,metacritic,...,added,reviews_count,background_image,dominant_color,saturated_color,esrb_rating_id,website,genre_ids,platform_ids,store_ids
0,1,dgeneration-hd,D/Generation HD,2015-10-23,False,1.86,1.0,6,1,NaN,...,128,7,https://media.rawg.io/media/screenshots/5c4/5c...,0f0f0f,0f0f0f,2.0,None,"[3, 7]","[1, 4, 5, 7, 18]","[1, 2, 3, 6]"
1,25,middle-earth-shadow-of-war,Middle-earth: Shadow of War,2017-09-27,False,3.86,4.0,1372,8,82.0,...,8701,1385,https://media.rawg.io/media/games/21a/21ad672c...,0f0f0f,0f0f0f,4.0,None,"[4, 5]","[1, 4, 18, 21]","[1, 2, 3, 4, 5, 8]"
2,28,red-dead-redemption-2,Red Dead Redemption 2,2018-10-26,False,4.59,5.0,5333,89,96.0,...,16533,5458,https://media.rawg.io/media/games/511/5118aff5...,0f0f0f,0f0f0f,4.0,None,[4],"[1, 4, 18]","[1, 2, 3, 11]"
3,30,wipeout-omega-collection,WipEout Omega Collection,2017-06-07,False,3.55,4.0,292,1,85.0,...,1682,293,https://media.rawg.io/media/screenshots/05d/05...,0f0f0f,0f0f0f,2.0,None,[1],[18],[3]
4,32,destiny-2,Destiny 2,2017-09-06,False,3.52,4.0,2652,21,82.0,...,14547,2678,https://media.rawg.io/media/games/34b/34b1f185...,0f0f0f,0f0f0f,3.0,None,"[2, 4]","[1, 4, 18, 171, 186, 187]","[1, 2, 3, 11]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3463,1009530,whisper-of-the-house,Whisper of the House,2025-08-27,False,0.00,0.0,2,0,NaN,...,7,2,https://media.rawg.io/media/screenshots/de5/de...,0f0f0f,0f0f0f,NaN,None,"[14, 40, 51]","[4, 5]",[1]
3574,1009662,cat-cat-house-room-escape,CAT CAT HOUSE : ROOM ESCAPE,2021-07-09,False,0.00,0.0,1,0,NaN,...,1,1,https://media.rawg.io/media/screenshots/9fe/9f...,0f0f0f,0f0f0f,NaN,None,"[40, 51]","[3, 21]",[None]
3731,1009841,snufkin-melody-of-moominvalley-fuddlers-courtship,Snufkin: Melody of Moominvalley - Fuddler's Co...,2024-12-17,False,0.00,0.0,1,0,NaN,...,1,1,https://media.rawg.io/media/screenshots/119/11...,0f0f0f,0f0f0f,NaN,None,"[3, 40]","[4, 7]",[None]
3940,1010070,whisker-wanderlust-the-mosaic-of-souls,Whisker Wanderlust: The Mosaic of Souls,2025-03-05,False,0.00,0.0,1,0,NaN,...,1,1,https://media.rawg.io/media/screenshots/1a0/1a...,0f0f0f,0f0f0f,NaN,None,"[40, 51]",[None],[None]


In [15]:
# Limpiar columnas
df = df.drop(columns=["slug", "released", "tba", "rating_top", "ratings_count", "reviews_text_count", "suggestions_count", "background_image",
                      "dominant_color", "saturated_color", "website", "reviews_count", "playtime"])

# Reemplazar NaN por la media y por rating pending
df['metacritic'] = df['metacritic'].fillna(df['metacritic'].mean())
df['esrb_rating_id'] = df['esrb_rating_id'].fillna(6.0)
# solo 2 decimales
df['metacritic'] = df['metacritic'].round(2)

df

,game_id,name,rating,metacritic,added,esrb_rating_id,genre_ids,platform_ids,store_ids
0,1,D/Generation HD,1.86,78.93,128,2.0,"[3, 7]","[1, 4, 5, 7, 18]","[1, 2, 3, 6]"
1,25,Middle-earth: Shadow of War,3.86,82.00,8701,4.0,"[4, 5]","[1, 4, 18, 21]","[1, 2, 3, 4, 5, 8]"
2,28,Red Dead Redemption 2,4.59,96.00,16533,4.0,[4],"[1, 4, 18]","[1, 2, 3, 11]"
3,30,WipEout Omega Collection,3.55,85.00,1682,2.0,[1],[18],[3]
4,32,Destiny 2,3.52,82.00,14547,3.0,"[2, 4]","[1, 4, 18, 171, 186, 187]","[1, 2, 3, 11]"
...,...,...,...,...,...,...,...,...,...
3463,1009530,Whisper of the House,0.00,78.93,7,6.0,"[14, 40, 51]","[4, 5]",[1]
3574,1009662,CAT CAT HOUSE : ROOM ESCAPE,0.00,78.93,1,6.0,"[40, 51]","[3, 21]",[None]
3731,1009841,Snufkin: Melody of Moominvalley - Fuddler's Co...,0.00,78.93,1,6.0,"[3, 40]","[4, 7]",[None]
3940,1010070,Whisker Wanderlust: The Mosaic of Souls,0.00,78.93,1,6.0,"[40, 51]",[None],[None]


In [16]:
df.info

<bound method DataFrame.info of       game_id                                               name  rating  \
0           1                                    D/Generation HD    1.86   
1          25                        Middle-earth: Shadow of War    3.86   
2          28                              Red Dead Redemption 2    4.59   
3          30                           WipEout Omega Collection    3.55   
4          32                                          Destiny 2    3.52   
...       ...                                                ...     ...   
3463  1009530                               Whisper of the House    0.00   
3574  1009662                        CAT CAT HOUSE : ROOM ESCAPE    0.00   
3731  1009841  Snufkin: Melody of Moominvalley - Fuddler's Co...    0.00   
3940  1010070            Whisker Wanderlust: The Mosaic of Souls    0.00   
3941  1010072                     Nintendo Switch 2 Welcome Tour    0.00   

      metacritic  added  esrb_rating_id     genre_ids  